In [1]:
# libraries
from dotenv import load_dotenv

# load environment variables from .env file
_ = load_dotenv("../../.env")

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langgraph.checkpoint.sqlite import SqliteSaver

In [3]:
class AgentState(TypedDict):
    lnode: str
    scratch: str
    count: Annotated[int, operator.add]

In [4]:
def node1(state: AgentState):
    print(f"node1, count:{state['count']}")
    return {
        "lnode": "node_1",
        "count": 1,
    }


def node2(state: AgentState):
    print(f"node2, count:{state['count']}")
    return {
        "lnode": "node_2",
        "count": 1,
    }


def should_continue(state):
    return state["count"] < 3

builder = StateGraph(AgentState)
builder.add_node("Node1", node1)
builder.add_node("Node2", node2)

builder.add_edge("Node1", "Node2")
builder.add_conditional_edges("Node2", should_continue, {True: "Node1", False: END})
builder.set_entry_point("Node1")

In [5]:
with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    graph = builder.compile(checkpointer=checkpointer)

In [6]:
thread = {"configurable": {"thread_id": str(1)}}
graph.invoke({"count": 0, "scratch": "hi"}, thread)

node1, count:0
node2, count:1
node1, count:2
node2, count:3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

In [7]:
graph.get_state(thread)

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed405-e407-6ab6-8004-1d2130098ddb'}}, metadata={'source': 'loop', 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}, 'step': 4}, created_at='2025-02-17T15:03:38.674859+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed405-e405-63d7-8003-fab8ccc553c9'}})

In [8]:
for state in graph.get_state_history(thread):
    print(state, "\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed405-e407-6ab6-8004-1d2130098ddb'}}, metadata={'source': 'loop', 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}, 'step': 4}, created_at='2025-02-17T15:03:38.674859+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed405-e405-63d7-8003-fab8ccc553c9'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed405-e405-63d7-8003-fab8ccc553c9'}}, metadata={'source': 'loop', 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}, 'step': 3}, created_at='2025-02-17T15:03:38.673864+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed405-e400-6596-8002-a0e8776c6c6f'}}) 

StateSnapshot(values={'lnode': 'node_2',

In [15]:
states = []
for state in graph.get_state_history(thread):
    states.append(state.config)
    print(state.config, state.values["count"])

{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed405-e407-6ab6-8004-1d2130098ddb'}} 4
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed405-e405-63d7-8003-fab8ccc553c9'}} 3
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed405-e400-6596-8002-a0e8776c6c6f'}} 2
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed405-e3f8-6191-8001-064c5ef0b8b6'}} 1
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed405-e3e6-67f2-8000-92054a79a229'}} 0
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed405-e3e1-69b3-bfff-f408cc1bff65'}} 0


In [14]:
states[-3]

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1efed405-e3f8-6191-8001-064c5ef0b8b6'}}

In [13]:
graph.get_state(states[-3])

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 1}, next=('Node2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed3fd-da40-6813-8001-6dccd3696c7c'}}, metadata={'source': 'loop', 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}, 'step': 1}, created_at='2025-02-17T15:00:02.901198+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed3fd-da25-6263-8000-0d37aefc82e8'}})

In [16]:
graph.invoke(None, states[-3])

node1, count:2
node2, count:3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

In [17]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state.config, state.values["count"])

{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed40b-f1b0-633f-8004-60bab848b4ab'}} 4
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed40b-f1ab-64d7-8003-2e3f43667eba'}} 3
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed40b-f1a5-69ef-8002-1b0d09d83328'}} 2
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed405-e407-6ab6-8004-1d2130098ddb'}} 4
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed405-e405-63d7-8003-fab8ccc553c9'}} 3
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed405-e400-6596-8002-a0e8776c6c6f'}} 2
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed405-e3f8-6191-8001-064c5ef0b8b6'}} 1
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed405-e3e6-67f2-8000-92054a79a229'}} 0
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkp

In [18]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state, "\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed40b-f1b0-633f-8004-60bab848b4ab'}}, metadata={'source': 'loop', 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}, 'step': 4}, created_at='2025-02-17T15:06:21.168313+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed40b-f1ab-64d7-8003-2e3f43667eba'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed40b-f1ab-64d7-8003-2e3f43667eba'}}, metadata={'source': 'loop', 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}, 'step': 3}, created_at='2025-02-17T15:06:21.166306+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efed40b-f1a5-69ef-8002-1b0d09d83328'}}) 

StateSnapshot(values={'lnode': 'node_2',

In [19]:
thread2 = {"configurable": {"thread_id": str(2)}}
graph.invoke({"count": 0, "scratch": "hi"}, thread2)

node1, count:0
node2, count:1
node1, count:2
node2, count:3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

In [20]:
states2 = []
for state in graph.get_state_history(thread2):
    states2.append(state.config)
    print(state.config, state.values["count"])

{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed40d-2d38-6a33-8004-429cd6bf5291'}} 4
{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed40d-2d33-6b46-8003-d681c3e57684'}} 3
{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed40d-2d2f-6fd7-8002-e90253e8e37e'}} 2
{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed40d-2d28-6ba8-8001-e2e72e6fa86d'}} 1
{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed40d-2d24-6c11-8000-de67331c4427'}} 0
{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed40d-2d1b-63ab-bfff-4a12c48c8aa7'}} 0


In [21]:
save_state = graph.get_state(states2[-3])
save_state

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 1}, next=('Node2',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed40d-2d28-6ba8-8001-e2e72e6fa86d'}}, metadata={'source': 'loop', 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}, 'step': 1}, created_at='2025-02-17T15:06:54.247824+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed40d-2d24-6c11-8000-de67331c4427'}})

In [25]:
save_state.values["count"] = -3
save_state.values["scratch"] = "***----***"
save_state

StateSnapshot(values={'lnode': 'node_1', 'scratch': '***----***', 'count': -3}, next=('Node2',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed40d-2d28-6ba8-8001-e2e72e6fa86d'}}, metadata={'source': 'loop', 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}, 'step': 1}, created_at='2025-02-17T15:06:54.247824+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed40d-2d24-6c11-8000-de67331c4427'}})

In [26]:
graph.update_state(thread2, save_state.values)

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1efed410-b6f4-6917-8005-f15e967b2bec'}}

In [28]:
thread = {"configurable": {"thread_id": str(2)}}
for state in graph.get_state_history(thread):
    print(state, "\n")

StateSnapshot(values={'lnode': 'node_1', 'scratch': '***----***', 'count': 1}, next=('Node1',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed410-b6f4-6917-8005-f15e967b2bec'}}, metadata={'source': 'update', 'step': 5, 'writes': {'Node2': {'lnode': 'node_1', 'scratch': '***----***', 'count': -3}}}, created_at='2025-02-17T15:08:29.227445+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed40d-2d38-6a33-8004-429cd6bf5291'}}) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed40d-2d38-6a33-8004-429cd6bf5291'}}, metadata={'source': 'loop', 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}, 'step': 4}, created_at='2025-02-17T15:06:54.254341+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed40d-2d33-6b46-8003-d681c3e57684'}}) 

Stat

In [31]:
graph.update_state(thread2, save_state.values, as_node="Node1")

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1efed417-ba68-6038-8006-cd1d754a40e1'}}

In [32]:
for i, state in enumerate(graph.get_state_history(thread2)):
    if i >= 3:  # print latest 3
        break
    print(state, "\n")

StateSnapshot(values={'lnode': 'node_1', 'scratch': '***----***', 'count': -2}, next=('Node2',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed417-ba68-6038-8006-cd1d754a40e1'}}, metadata={'source': 'update', 'step': 6, 'writes': {'Node1': {'lnode': 'node_1', 'scratch': '***----***', 'count': -3}}}, created_at='2025-02-17T15:11:37.494124+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed410-b6f4-6917-8005-f15e967b2bec'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': '***----***', 'count': 1}, next=('Node1',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed410-b6f4-6917-8005-f15e967b2bec'}}, metadata={'source': 'update', 'step': 5, 'writes': {'Node2': {'lnode': 'node_1', 'scratch': '***----***', 'count': -3}}}, created_at='2025-02-17T15:08:29.227445+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1e

In [33]:
graph.invoke(None, thread2)

node2, count:-2
node1, count:-1
node2, count:0
node1, count:1
node2, count:2


{'lnode': 'node_2', 'scratch': '***----***', 'count': 3}

In [34]:
for state in graph.get_state_history(thread2):
    print(state, "\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': '***----***', 'count': 3}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed418-90bb-62c4-800b-b30c7015d69e'}}, metadata={'source': 'loop', 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}, 'step': 11}, created_at='2025-02-17T15:11:59.967712+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed418-90b8-6bb5-800a-5e6ac213213c'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': '***----***', 'count': 2}, next=('Node2',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed418-90b8-6bb5-800a-5e6ac213213c'}}, metadata={'source': 'loop', 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}, 'step': 10}, created_at='2025-02-17T15:11:59.966712+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efed418-90b3-6d58-8009-08a35291aed9'}}) 

StateSnapshot(values={